In [1]:
# from confluent_kafka import Consumer, Producer, KafkaError, KafkaException
import sys
from multiprocessing import shared_memory
import numpy as np
from scipy.io import wavfile 
import time
from IPython.display import Audio
from utils.shared_memory_queue import SharedMemoryWithIndexes

In [57]:
from kafka import TopicPartition, KafkaConsumer

# Create the consumer
consumer = KafkaConsumer(bootstrap_servers='localhost:9092',
                         auto_offset_reset='earliest',  # Start consuming from the earliest available message
                         enable_auto_commit=False)       # Disable auto-commit of offsets

# Manually assign a partition
topic_partition = TopicPartition('tts_messages', 0)  # Assign partition 0
consumer.assign([topic_partition])

messages = [0] * 100000
for msg in consumer:
    messages[msg.offset] = msg.value
    msg.timestamp
    if msg.offset == 99999:
        break


In [5]:
from kafka import KafkaProducer
# config = {
#     "bootstrap_servers": "localhost:9092",
#     # 'client.id': socket.gethostname()
# }

producer = KafkaProducer(bootstrap_servers='localhost:9092')
print(producer.bootstrap_connected())


True


In [202]:
producer.flush()


In [8]:
import json
num = 1000
objects = [{
        "message":"Today I was a given a task to build a system that can process audio files and convert them to text. I have a lot of experience in this area, having worked on similar projects in the past. I am confident that I can handle this task and deliver a high-quality solution.", 
        "language":"en", 
        "voice":"female",
        "speed":1.0,
        "shm_name":f"shm_{i}",
        "id":i
    } for i in range(num)]


objects_json = [json.dumps(obj) for obj in objects]
objects_bytes = [obj.encode('utf-8') for obj in objects_json]

In [128]:
# create a shm

try:
    shm = shared_memory.SharedMemory(name=objects[0]["shm_name"], create=True, size=7158805 * 2)
except Exception as e:
    shm = shared_memory.SharedMemory(name=objects[0]["shm_name"])
shm.size


14319616

In [16]:

for i in range(5,10):
    # create a shared memory 
    try:
        shm = shared_memory.SharedMemory(name=objects[i]["shm_name"], create=True, size=7158805 * 2)
    except Exception as e:
        pass
    producer.send('tts_messages', value=objects_bytes[i])


In [17]:
shm_queue = SharedMemoryWithIndexes("shm_9")

Exception ignored in: <function SharedMemory.__del__ at 0x122905940>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/shared_memory.py", line 187, in __del__
    self.close()
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/shared_memory.py", line 230, in close
    self._mmap.close()
BufferError: cannot close exported pointers exist


In [18]:
shm_queue.free_memory_index

array([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

In [19]:
shm_queue.read_index

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

In [3]:
rate, audio_chunk = wavfile.read("../audios/go_new.wav")
# print(audio_chunk[0:10])
# get only first 48000 samples


In [194]:
audio_chunk.shape

(7158805,)

In [21]:
Audio(shm_queue.dequeue(), rate=rate)

1


In [196]:
arrays=  []
audio_chunks = []
for i in range(10):
    size = 96000
    start = i * size
    ending = (i+1) * size 
    print(start, ending)
    temp = audio_chunk[start:ending]
    audio_chunks.append(temp)
    # temp = temp.astype(np.int16)
    shm_queue.enqueue(temp)
    arrays.append(shm_queue.dequeue())

0 96000
0
0
96000 192000
1
1
192000 288000
2
2
288000 384000
3
3
384000 480000
4
4
480000 576000
5
5
576000 672000
6
6
672000 768000
7
7
768000 864000
8
8
864000 960000
9
9
